In [4]:
import pandas as pd

In [19]:
df = pd.read_csv(r"C:\Users\gusti\Notebooks\urgencias\AtencionesUrgenciaLineal2009.csv")

In [7]:
df[df["semana"]==1]["fecha"].unique()

array(['2009-01-04', '2009-01-05', '2009-01-06', '2009-01-07',
       '2009-01-08', '2009-01-09', '2009-01-10'], dtype=object)

In [20]:
df["Año"] = df["fecha"].apply(lambda x: str(x)[0:4])

In [22]:
del df["Menores de 1 año"]
del df["1-4 años"]
del df["5-14 años"]
del df["15-64 años"]
del df["65 años y más"]

In [24]:
df2 = df.groupby(by=['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa',
                     'semana', 'GLOSATIPOESTABLECIMIENTO',
                     'GLOSATIPOATENCION', 'GlosaTipoCampana', 'Año']).sum()

In [25]:
df2 = df2.reset_index()
#df2.to_excel("PruebaUrgencias2009.xlsx", index = False)

In [26]:
dfRef = pd.read_excel(r"C:\Users\gusti\Notebooks\RefUrgencias.xlsx")

In [27]:
dfRef2 = pd.read_excel(r"C:\Users\gusti\Notebooks\RefRegionUrgencias.xlsx")

In [28]:
dfExcluidos = pd.read_excel(r"C:\Users\gusti\Notebooks\RefUrgencias.xlsx", sheet_name = "Excluidos")

In [33]:
import warnings
warnings.filterwarnings('ignore')
dfReferencia = df[['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
                   'GlosaTipoCampana']]
dfReferencia2 = dfReferencia.drop_duplicates()
df2_1 = df2[df2["semana"]==df2["semana"].unique()[0]]
df2_1[df2["semana"].unique()[0]] = df2_1["Total"]
del df2_1["Total"]
del df2_1["semana"]
df2_1 = df2_1.merge(dfReferencia2, left_on=['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana'], right_on=['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana'], how="right")
for i in df2["semana"].unique()[1:]:
    dfAux = df2[df2["semana"] == i]
    dfAux[i] = dfAux["Total"]
    del dfAux["semana"]
    del dfAux["Total"]
    df2_1 = df2_1.merge(dfAux, left_on=['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana','Año'], right_on= ['IdEstablecimiento', 'NEstablecimiento', 'IdCausa', 'GlosaCausa', 'GLOSATIPOESTABLECIMIENTO', 'GLOSATIPOATENCION',
       'GlosaTipoCampana','Año'],how="left")

In [34]:
for i in df2_1.columns:
    print(i)

IdEstablecimiento
NEstablecimiento
IdCausa
GlosaCausa
GLOSATIPOESTABLECIMIENTO
GLOSATIPOATENCION
GlosaTipoCampana
Año
13
25
26
27
28
29
30
31
39
40
41
42
43
44
45
46
47
48
49
50
51
52
32
33
34
35
36
37
38
1
2
3
4
5
6
7
8
9
10
11
12
14
15
16
17
18
19
20
21
22
23
24


In [35]:
dfFix2 = df2_1[['GlosaCausa',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
                26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,
                50,51,52,'Año','GLOSATIPOESTABLECIMIENTO','NEstablecimiento','GLOSATIPOATENCION']]

In [36]:
dfFix3 = dfFix2.rename(columns={'Año':'Fecha','GLOSATIPOESTABLECIMIENTO':'Tipo Establecimiento',
                                'NEstablecimiento':'Nombre establecimiento','GLOSATIPOATENCION':'Tipo'})

In [37]:
excluidos = list(dfExcluidos["Excluidos"])

In [38]:
dfFix4 = dfFix3[dfFix3["GlosaCausa"].apply(lambda x: x not in excluidos)]

In [39]:
dfMerge = dfFix4.merge(dfRef, left_on= "GlosaCausa", right_on= "Urgencia", how= "inner")

In [40]:
dfMergeFix = dfMerge[['Urgencia',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,
                      27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,
                      49,50,51,52,'Fecha','Tipo Establecimiento','Nombre establecimiento',
                      'Tipo','Causas']]

In [41]:
dfFinal = dfMergeFix.merge(dfRef2, left_on= "Nombre establecimiento", right_on= "Nombre establecimiento", how= "inner")

In [42]:
dfFinal2 = dfFinal[['Urgencia',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,
                      27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,
                      49,50,51,52,'Fecha','Region','Tipo Establecimiento','Nombre establecimiento',
                      'Tipo','Causas']]

In [43]:
dfFinal2.to_excel("Urgencias2009.xlsx", index = False)